In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, '../')

import fewshot.data
import fewshot.trainer

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models

import matplotlib.pyplot as plt
%matplotlib inline

# Moving on to large dataset
We've refactored our data pipeline, utilites and training loop in the `fewshot` module. Let's test it below for a full run of training.

In [3]:
fewshot.data.fix_quotes('../data/fashion-dataset/styles.csv', '../data/fashion-dataset/styles_quoted.csv')

In [4]:
train_transform = transforms.Compose([
        transforms.Resize(300),
        transforms.RandomCrop(220),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

test_transform = transforms.Compose([
        transforms.Resize(300),
        transforms.CenterCrop(220),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

In [5]:
data = fewshot.data.FashionData('../data/fashion-dataset/styles_quoted.csv',
                                '../data/fashion-dataset/images/',
                               train_transform=train_transform,
                               test_transform=test_transform,
                               top20=True)

In [6]:
n_classes = len(data.idx2name)
n_classes

20

In [7]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features

## replacing the last layer with the dense layer we need
model.fc = nn.Linear(num_ftrs, n_classes)

In [8]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
fewshot.trainer.run_training(model, 5, data, optimizer, loss_func, gpu_id=1, root='./traces/')

------------- epoch:  0
Training loss: 0.741
Validation loss: 0.963
Validation accuracy: 0.782
------------- epoch:  1
Training loss: 0.449
Validation loss: 1.311
Validation accuracy: 0.732
------------- epoch:  2
Training loss: 0.376


KeyboardInterrupt: 

Looks like this works. To do:
- change to top1 and top5 accuracy results
- change the data to be able to switch between the top20 and the non-top20 classes

In [ ]:
checkpoint = torch.load('./gpu_0_Jun07_1450/model_epoch_9.pth')

In [ ]:
model.load_state_dict(checkpoint['model_state_dict'])
model.cuda()

In [ ]:
test_dl = torch.utils.data.DataLoader(data.test_ds,
                                    batch_size=64*2, shuffle=True,
                                    num_workers=4)

In [ ]:
images, labels = next(iter(test_dl))

In [ ]:
images.shape

In [ ]:
output = model(images.cuda())

In [ ]:
output.shape

In [ ]:
_, pred = output.topk(5, 1, True, True)

In [ ]:
pred = pred.t()

In [ ]:
pred.shape

In [ ]:
labels = torch.tensor([data.name2idx[label] for label in labels]).cuda()

In [ ]:
labels.view(1, -1).shape

In [ ]:
 correct = pred.eq(labels.view(1, -1).expand_as(pred))

In [ ]:
correct

In [ ]:
correct1 = correct[:1].view(-1).float()

In [ ]:
correct1

In [ ]:
correct5 = correct[:5].sum(0)

In [ ]:
correct5.sum()

In [ ]:
correct1.sum()

In [ ]:
correct_all = torch.zeros(n_classes).cuda()

In [ ]:
correct_all[labels] += correct1

In [ ]:
correct_all

In [ ]:
correct1

In [ ]:
labels

In [ ]:
correct5

In [ ]:
c1, c5 = fewshot.trainer.evaluate(model, data, 0, n_classes)

In [ ]:
c1

In [ ]:
c5